#### Make sure to install the following packages first.
(Otherwise the codes won't run) 

- pip install customtkinter
- pip install Pillow
- pip install sklearn
- pip install pandas
  

In [1]:
import customtkinter as ctk
from tkinter import *
from tkinter import Tk, font
from tkinter import filedialog
from PIL import Image, ImageTk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import io
import requests

In [2]:
ctk.set_appearance_mode("light")
#styling
offwhite = "#F9F7F3"
pressed_offwhite = "#FCF7EC"

In [3]:
import pickle
import pandas as pd
# with open("combined_features_books_final.pkl", "rb") as f:
#     books = pickle.load(f)
#     book_titles = list(books["title"])

#changed the data frame reading

books = pd.read_csv("books_finaldata.csv")
book_titles = list(books["title"])
#books.head()

In [ ]:
#main window
main = ctk.CTk()
main.title("Books!")
main.configure(fg_color=offwhite)
main.geometry("1440x1024+0+0")
main.minsize(1440,1024)
main.grid_columnconfigure(0, weight=1) 
main.grid_columnconfigure(1, weight=0)  
#left side
left = ctk.CTkFrame(
    main,
    height = 1024,
    width = 720,
    fg_color = offwhite,
    corner_radius = 0
)
left.grid(row=0, column=0, sticky="nsew")
left.grid_columnconfigure(0, weight=1) 


#title/welcome text
welcome = ctk.CTkLabel(
    left,
    text = "Let's get to reading!",
    font = ("Times", 50)
)
welcome.grid(row=0, column=0, pady=35)

#instructions
instructions = ctk.CTkLabel(
    left,
    text="Upload a txt file (☝︎) or enter a book you enjoy in the search box below.\nWe will give you recommendations for your next read.",
    text_color="black",
    font=("Times", 20)
)
instructions.grid(row=1, column=0)

#buttons frame
button_frame = ctk.CTkFrame(
    left,
    height = 75,
    width = 375,
    fg_color = offwhite,
    corner_radius = 0
)
button_frame.grid(row=2, column=0, pady=25)

# Define global variables
user_book_titles = []

# Function to allow user to delete books from their list
def delete_pressed():
    selected_items = user_books.curselection()
    for index in selected_items[::-1]:
        # Remove the title from the user_book_titles list
        removed_title = user_books.get(index)
        user_book_titles.remove(removed_title)
        # Delete the item from the listbox
        user_books.delete(index)

# Function to display the user's books in the listbox
def display_user_books():
    user_books.delete(0, END)  # Clear existing entries
    for title in user_book_titles:
        user_books.insert(END, title)

# Function to handle uploading of books from a file
def upload_pressed():
    global user_book_titles

    # Ask user to select a file for uploading
    filename = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
    if filename:
        # Open the selected file for reading
        with open(filename, 'r') as f:
            # Read each line from the file and add it to the user_book_titles list
            user_book_titles = [line for line in f]

        # Display the user's books in the listbox
        display_user_books()

# Buttons frame
button_frame = ctk.CTkFrame(
    left,
    height=75,
    width=375,
    fg_color=offwhite,
    corner_radius=0
)
button_frame.grid(row=2, column=0, pady=25)

# Upload button
upload_button = ctk.CTkButton(
    button_frame,
    width=60,
    height=60,
    corner_radius=50,
    text="☝︎",
    text_color="black",
    font=("Times", 60),
    fg_color="grey",
    command=upload_pressed
)
upload_button.grid(row=0, column=0, padx=30)

#my books frame
mybooks_frame = ctk.CTkFrame(
    left,
    height=20,
    width=575,
    fg_color="white"
)
mybooks_frame.grid(row=4, column=0, columnspan=2, sticky="ew", padx=20)
mybooks_frame.grid_columnconfigure(0, weight=1) 


# Listbox to display user's books
user_books = Listbox(
    mybooks_frame,
    font=("Times", 20),
    bd=0
)
user_books.grid(row=1, column=0, columnspan=2, sticky="nsew")  

# My books label
my_books_label = ctk.CTkLabel(
    mybooks_frame,
    text="My booklist:",
    text_color="black",
    font=("Times", 20, "italic")
)
my_books_label.grid(row=0, column=0)

# Delete button
delete_button = ctk.CTkButton(
    mybooks_frame,
    text="⌦",
    width=40,
    height=40,
    corner_radius=40,
    text_color="black",
    font=("Times", 30),
    fg_color="white",
    command=delete_pressed
)
delete_button.grid(row=0, column=1)

#save button
def save_pressed():
    filename = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
    if filename:
        with open(filename, 'w') as f:
            for title in user_book_titles:
                f.write(title + "\n")
                
#save
save = ctk.CTkButton(
    button_frame, 
    width=81, 
    height=81, 
    corner_radius=40, 
    text="✍︎",
    text_color="black",
    font=("Times", 60),
    command=save_pressed
)
save.grid(row=0, column=2, padx=30)



#right side
right = ctk.CTkScrollableFrame(
    main,
    height = 2000,
    width = 720,
    fg_color = "white",
    corner_radius = 0
)
right.grid(row=0, column=1, sticky="nsew")
right.grid_columnconfigure(0, weight=1) 
#search bar
#frame
search_frame = ctk.CTkFrame(
    left,
    height = 75,
    width = 375,
    fg_color = offwhite,
    corner_radius = 0
)
search_frame.grid(row=3, column=0, pady=25)

#autocomplete adopted from https://www.geeksforgeeks.org/autocmplete-combobox-in-python-tkinter/   
# Function for checking the key pressed and updating the listbox 
l = book_titles
def checkkey(event): 
    value = event.widget.get() 

    # get data from l 
    if value == '': 
        data = l 
    else: 
        data = [] 
        for item in l: 
            if value.lower() in item.lower(): 
                data.append(item)				 

    # update data in listbox 
    update(data) 

def update(data): 
    # clear previous data 
    lb.delete(0, 'end') 
    # put new data 
    for item in data: 
        lb.insert('end', item) 

# Function to handle listbox selection event
def on_select(event):
    # Get the selected item
    selected_item_index = lb.curselection()
    if selected_item_index:
        selected_item = lb.get(selected_item_index[0])
        # Set the entry text to the selected item
        e.delete(0, 'end')
        e.insert('end', selected_item)

#creating text box 
e = Entry(
    search_frame,
    font=("Times", 20)
) 
e.grid(row=0, column=0) 
e.bind('<KeyRelease>', checkkey) 

#creating list box 
lb = Listbox(
    search_frame,
    font=("Times", 20),
    height=5,

    bd=0
) 
lb.grid(row=1, column=0) 
lb.bind('<<ListboxSelect>>', on_select)
update(l) 

#book results frame
book_results_frame = ctk.CTkFrame(
    right,
    fg_color = offwhite,
    height = 2000,
    width = 650
)
book_results_frame.grid(row=1)

# Function to load an image from a URL (in our case the book covers)
def load_image(url):
    response = requests.get(url)
    img_data = response.content
    img = Image.open(io.BytesIO(img_data))
    img_tk = ImageTk.PhotoImage(img)
    return img_tk

# Function to create book buttons based on search results
def create_book_buttons(book_title_results, book_author_results, book_thumbnail_results, book_year_results):
    colors = ["#D37676", "#D3B976", "#94B49F"]
    for i, title in enumerate(book_title_results):
        # Calculate the row and column for each book button
        row_num = i // 3
        col_num = i % 3
        color = colors[col_num] 
        # Create a button for each book title
        # If a thumbnail exists we make a button with an image, 
        # otherwise we create one with just the text: title, published year, and author
        if isinstance(book_thumbnail_results[i], str):
            cover_img = load_image(book_thumbnail_results[i])
            # The customtkinter library doesn't import images in buttons, 
            # so we use the standard tkinter function for a button here
            book_button = Button(
                book_results_frame,
                text=title + " (" + book_year_results[i] + ")" + "\n\n- " + book_author_results[i],
                image = cover_img,
                height = 320,
                width = 200,
                font=("Times", 20),
                command=lambda t=title: book_button_pressed(t)  # Pass book title as argument -> will say __ book was pressed
            )
            book_button.image = cover_img
        else:
            book_button = ctk.CTkButton(
                book_results_frame,
                text=title + " (" + book_year_results[i] + ")" + "\n\n- " + book_author_results[i],
                height = 320,
                width = 200,
                fg_color=color,
                font=("Times", 20),
                hover = False,
                command=lambda t=title: book_button_pressed(t)  # Pass book title as argument -> will say __ book was pressed
            )
        #why lambda was used -> https://www.freecodecamp.org/news/python-lambda-function-explained/#:~:text=lambda%20is%20a%20keyword%20in,what%20you%20want%20to%20achieve.
        book_button.grid(row=row_num, column=col_num, pady=10, padx=10)
        
# Function to handle book button presses
def book_button_pressed(title):
    cleaned_title = " ".join(title.split())
    user_book_titles.append(cleaned_title)
    display_user_books()

#function to display what they searched for
def submit_search():
    #clearing previous results
    for widget in book_results_frame.winfo_children():
        widget.destroy()
    results.configure(text=f"Similar books to {e.get()}")
    #altered code from notebook recommender system
    def results_book(insert_book_name):
        insert_book_name =insert_book_name.lower()    
        tfidf = TfidfVectorizer(stop_words='english')
        tfidf_matrix = tfidf.fit_transform(books['combined_features'])
        cosine_sim = cosine_similarity(tfidf_matrix)

        indices = pd.Series(books.index, index = books["title_lower"])
        index = indices[insert_book_name]
        sim_scores = list(enumerate(cosine_sim[index])) 
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:7]
        books_indices = [i[0] for i in sim_scores]
        results = books.iloc[books_indices]
        final_results = results[["title", "authors", "description", "thumbnail", "published_year"]]
        titles = final_results["title"]
        authors = final_results["authors"]
        thumbnails = final_results["thumbnail"]
        published_year = final_results["published_year"]
        
        return titles, authors, thumbnails, published_year
    
    returned_titles, returned_authors, returned_thumbnails, returned_published_year = results_book(e.get())
    
     #split titles that are too long
    def split_title(list_title, length):
        final_list = []
        for title in list_title:
            if len(title) > length:
                title_split = title.split()
                for index,split in enumerate(title_split):
                    if index%2 == 1:
                        title_split.insert(index, "\n")
                final_list.append(" ".join(title_split))
            else:
                final_list.append(title)
        return final_list
    # Converting all the data into the right format, so they can be used in the button function
    search_results_titles = split_title(list(returned_titles), 15)
    search_results_authors = split_title(list(returned_authors), 15)
    search_results_thumbnails = list(returned_thumbnails)
    search_results_year = list(returned_published_year.astype(int).astype(str))
    create_book_buttons(search_results_titles, search_results_authors, search_results_thumbnails, search_results_year)

#submit button for search
submit_search_button = ctk.CTkButton(
    search_frame,
    text="Search",
    text_color="black",
    font=("Times", 20),
    command=submit_search
)
submit_search_button.grid(row=0, column=1, pady=5)

#results frame
results_frame = ctk.CTkFrame(
    right,
    fg_color = "white",
    corner_radius = 0
)
results_frame.grid(row=0, column=0)

#display what they searched for
results = ctk.CTkLabel(
    results_frame,
    text="",
    text_color="black",
    font= ("Times", 30)
)
results.grid(row=0, column=0, pady=35)
    

main.mainloop()